# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
import holoviews as hv
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,84,2,1.34,NZ,1699786522
1,1,tura,25.5198,90.2201,21.85,83,2,1.00,IN,1699786523
2,2,palikir - national government center,6.9248,158.1611,25.41,94,100,1.54,FM,1699786524
3,3,bethel,41.3712,-73.4140,1.05,81,40,0.00,US,1699786525
4,4,hermanus,-34.4187,19.2345,17.85,58,58,3.63,ZA,1699786526


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600
)


# Display the map
map_plot_1

ImportError: Geographic projection support requires: cartopy, geoviews.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = -20
ideal_temp_max = 35
ideal_humi_max = 90
ideal_clouds = 100
ideal_wind_max = 10

ideal_city_data_df = city_data_df.loc[(city_data_df['Max Temp']>=ideal_temp_min) &
                                    (city_data_df['Max Temp']<=ideal_temp_max) &
                                    (city_data_df['Humidity']<=ideal_humi_max) &
                                    (city_data_df['Cloudiness']<=ideal_clouds) &
                                    (city_data_df['Wind Speed']<=ideal_wind_max),:]




# Drop any rows with null values
clean_df = ideal_city_data_df.dropna()
# Display sample data
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,84,2,1.34,NZ,1699786522
1,1,tura,25.5198,90.2201,21.85,83,2,1.00,IN,1699786523
3,3,bethel,41.3712,-73.4140,1.05,81,40,0.00,US,1699786525
4,4,hermanus,-34.4187,19.2345,17.85,58,58,3.63,ZA,1699786526
5,5,georgetown,5.4112,100.3354,30.97,70,20,2.57,MY,1699786527
...,...,...,...,...,...,...,...,...,...,...
560,560,monduli,-3.3000,36.4500,23.22,63,89,1.91,TZ,1699787136
562,562,vanino,49.0869,140.2543,-10.04,66,4,4.50,RU,1699787138
564,564,chela,34.0690,104.4619,1.85,45,100,1.80,CN,1699787140
567,567,coahuayana de hidalgo,18.7000,-103.6583,24.21,88,58,1.66,MX,1699787143


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,waitangi,NZ,12.12,-43.9535,-176.5597,
1,tura,IN,21.85,25.5198,90.2201,
3,bethel,US,1.05,41.3712,-73.4140,
4,hermanus,ZA,17.85,-34.4187,19.2345,
5,georgetown,MY,30.97,5.4112,100.3354,
8,ilulissat,GL,-6.99,69.2167,-51.1000,
10,olonkinbyen,SJ,-2.77,70.9221,-8.7187,
11,yeniseysk,RU,-5.03,58.4497,92.1703,
14,papatowai,NZ,12.92,-46.5619,169.4708,
15,west island,CC,27.99,-12.1568,96.8225,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"

# Search radius is 10km
radius = 10000

# Return only one item
limit = 1

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Set base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,'Lng']
    latitude = hotel_df.loc[index,'Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Make and API request using the params dictionary and convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
display(hotel_df)

Starting hotel search
waitangi - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
yeniseysk - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
brignoles - nearest hotel: No hotel found
prince rupert - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
mersa matruh - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
puerto armuelles - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
vandergrift - nearest hotel: No hotel found
leitchfield - neare

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,waitangi,NZ,12.12,-43.9535,-176.5597,No hotel found
1,tura,IN,21.85,25.5198,90.2201,No hotel found
3,bethel,US,1.05,41.3712,-73.4140,No hotel found
4,hermanus,ZA,17.85,-34.4187,19.2345,No hotel found
5,georgetown,MY,30.97,5.4112,100.3354,No hotel found
...,...,...,...,...,...,...
560,monduli,TZ,23.22,-3.3000,36.4500,No hotel found
562,vanino,RU,-10.04,49.0869,140.2543,No hotel found
564,chela,CN,1.85,34.0690,104.4619,No hotel found
567,coahuayana de hidalgo,MX,24.21,18.7000,-103.6583,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = 'City',
    size = "Humidity",
    scale = 1.5,
    hover_cols=['Hotel Name', 'Country'],
)


# Display the map
map_plot_3

ImportError: Geographic projection support requires: cartopy, geoviews.